In [30]:
import os
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, r'eye-gaze\eye-gaze')
if not os.path.exists(final_directory):
   os.makedirs(final_directory)

In [32]:
import zipfile
with zipfile.ZipFile("eye-gaze\eye-gaze.zip","r") as zip_ref:
    zip_ref.extractall("eye-gaze\eye-gaze")

In [ ]:
!pip install scikit-image==0.16.2

In [27]:
import os
import sys
import keras
from keras import applications
from keras import layers
from keras import models
from keras import optimizers
from keras.layers import add
from keras.preprocessing import image
import h5py
import numpy as np
import tensorflow as tf

In [28]:
from platform import python_version

print(python_version())

3.8.15


In [29]:
print('tf-version',tf.__version__, 'keras-version', keras.__version__)

tf-version 2.9.2 keras-version 2.9.0


Loading Data
Here we setup the paths and load the data from the hdf5 files since there would otherwise be too many individual jpg/png images

In [43]:
path = os.path.dirname(os.path.abspath('.'))
data_dir = os.path.join(r'C:\\Users\\smoha\\Downloads\\Bio_informatics_final_project\\eye-gaze\\eye-gaze\\')
cache_dir = '.'

# load the data file and extract dimensions
with h5py.File(os.path.join(data_dir,'gaze.h5'),'r') as t_file:
    print(list(t_file.keys()))
    assert 'image' in t_file, "Images are missing"
    assert 'look_vec' in t_file, "Look vector is missing"
    assert 'path' in t_file, "Paths are missing"
    print('Synthetic images found:',len(t_file['image']))
    for _, (ikey, ival) in zip(range(1), t_file['image'].items()):
        print('image',ikey,'shape:',ival.shape)
        img_height, img_width = ival.shape
        img_channels = 1
    syn_image_stack = np.stack([np.expand_dims(a,-1) for a in t_file['image'].values()],0)

with h5py.File(os.path.join(data_dir,'real_gaze.h5'),'r') as t_file:
    print(list(t_file.keys()))
    assert 'image' in t_file, "Images are missing"
    print('Real Images found:',len(t_file['image']))
    for _, (ikey, ival) in zip(range(1), t_file['image'].items()):
        print('image',ikey,'shape:',ival.shape)
        img_height, img_width = ival.shape
        img_channels = 1
    real_image_stack = np.stack([np.expand_dims(a,-1) for a in t_file['image'].values()],0)

#
# training params
#

nb_steps = 10000 #20 # originally 10000, but this makes the kernel time out
batch_size = 49
k_d = 1  # number of discriminator updates per step
k_g = 2  # number of generative network updates per step
log_interval = 100
pre_steps = 15 # for pretraining

['image', 'look_vec', 'path']
Synthetic images found: 50000
image ..\..\..\..\Downloads\UnityEyes_Windows\UnityEyes_Windows\imgs\1.jpg shape: (35, 55)
['image']
Real Images found: 34100
image ..\data\MPIIGaze_Dataset\000055ed-890a-411f-be6e-b8453b495389.png shape: (35, 55)


In [ ]:
import skimage
print(skimage.__version__)

Utility Functions: These functions make it a bit easier to keep track of how the training is going and make it

In [ ]:
"""
Module to plot a batch of images along w/ their corresponding label(s)/annotations and save the plot to disc.

Use cases:
Plot images along w/ their corresponding ground-truth label & model predicted label,
Plot images generated by a GAN along w/ any annotations used to generate these images,
Plot synthetic, generated, refined, and real images and see how they compare as training progresses in a GAN,
etc...
"""
import matplotlib
from matplotlib import pyplot as plt

from itertools import groupby
from skimage.util import montage as montage2d
def plot_batch(image_batch, figure_path, label_batch=None):
    
    all_groups = {label: montage2d(np.stack([img[:,:,0] for img, lab in img_lab_list],0)) 
                  for label, img_lab_list in groupby(zip(image_batch, label_batch), lambda x: x[1])}
    fig, c_axs = plt.subplots(1,len(all_groups), figsize=(len(all_groups)*4, 8), dpi = 600)
    for c_ax, (c_label, c_mtg) in zip(c_axs, all_groups.items()):
        c_ax.imshow(c_mtg, cmap='bone')
        c_ax.set_title(c_label)
        c_ax.axis('off')
    fig.savefig(os.path.join(figure_path))
    plt.close()
"""
Module implementing the image history buffer described in `2.3. Updating Discriminator using a History of
Refined Images` of https://arxiv.org/pdf/1612.07828v1.pdf.

"""
class ImageHistoryBuffer(object):
    def __init__(self, shape, max_size, batch_size):
        """
        Initialize the class's state.

        :param shape: Shape of the data to be stored in the image history buffer
                      (i.e. (0, img_height, img_width, img_channels)).
        :param max_size: Maximum number of images that can be stored in the image history buffer.
        :param batch_size: Batch size used to train GAN.
        """
        self.image_history_buffer = np.zeros(shape=shape)
        self.max_size = max_size
        self.batch_size = batch_size

    def add_to_image_history_buffer(self, images, nb_to_add=None):
        """
        To be called during training of GAN. By default add batch_size // 2 images to the image history buffer each
        time the generator generates a new batch of images.

        :param images: Array of images (usually a batch) to be added to the image history buffer.
        :param nb_to_add: The number of images from `images` to add to the image history buffer
                          (batch_size / 2 by default).
        """
        if not nb_to_add:
            nb_to_add = self.batch_size // 2

        if len(self.image_history_buffer) < self.max_size:
            np.append(self.image_history_buffer, images[:nb_to_add], axis=0)
        elif len(self.image_history_buffer) == self.max_size:
            self.image_history_buffer[:nb_to_add] = images[:nb_to_add]
        else:
            assert False

        np.random.shuffle(self.image_history_buffer)

    def get_from_image_history_buffer(self, nb_to_get=None):
        """
        Get a random sample of images from the history buffer.

        :param nb_to_get: Number of images to get from the image history buffer (batch_size / 2 by default).
        :return: A random sample of `nb_to_get` images from the image history buffer, or an empty np array if the image
                 history buffer is empty.
        """
        if not nb_to_get:
            nb_to_get = self.batch_size // 2

        try:
            return self.image_history_buffer[:nb_to_get]
        except IndexError:
            return np.zeros(shape=0)

Network Architectures: Here we define the two primary networks which are refiner network and discriminator network. They work against each other to constantly improve the results of the other one.

In [ ]:
def refiner_network(input_image_tensor):
    """
    The refiner network, Rθ, is a residual network (ResNet). It modifies the synthetic image on a pixel level, rather
    than holistically modifying the image content, preserving the global structure and annotations.

    :param input_image_tensor: Input tensor that corresponds to a synthetic image.
    :return: Output tensor that corresponds to a refined synthetic image.
    """
    def resnet_block(input_features, nb_features=64, nb_kernel_rows=3, nb_kernel_cols=3):
        """
        A ResNet block with two `nb_kernel_rows` x `nb_kernel_cols` convolutional layers,
        each with `nb_features` feature maps.

        See Figure 6 in https://arxiv.org/pdf/1612.07828v1.pdf.

        :param input_features: Input tensor to ResNet block.
        :return: Output tensor from ResNet block.
        """
        y = tf.keras.layers.Conv2D(nb_features, (nb_kernel_rows, nb_kernel_cols),activation='relu', padding ='same')(input_features)
        print(y.shape)
        
        y =tf.keras.layers.Conv2D(nb_features, (nb_kernel_rows, nb_kernel_cols),activation='relu', padding ='same')(y)
        print(y.shape)
        y = keras.layers.add([input_features, y])        
        #y = layers.merge([input_features, y], mode='sum')
        return layers.Activation('relu')(y)

    # an input image of size w × h is convolved with 3 × 3 filters that output 64 feature maps
    print("size of input shape")
    print(input_image_tensor.shape)
    x = tf.keras.layers.Conv2D(64, (3,3),  padding='same', activation='relu')(input_image_tensor)
    print("size of first")
    print(x.shape)

    # the output is passed through 4 ResNet blocks
    for _ in range(4):
        x = resnet_block(x)
        print("size after interation")
        print(x.shape)

    #x = layers.Convolution2D(img_channels, 1, 1, padding='same', activation='tanh')(x)
    #print("size before sending")
    #print(x.shape)
    # the output of the last ResNet block is passed to a 1 × 1 convolutional layer producing 1 feature map
    # corresponding to the refined synthetic image
    return layers.Convolution2D(img_channels, 1, 1, padding='same', activation='tanh')(x)

In [ ]:

def discriminator_network(input_image_tensor):
    """
    The discriminator network, Dφ, contains 5 convolution layers and 2 max-pooling layers.

    :param input_image_tensor: Input tensor corresponding to an image, either real or refined.
    :return: Output tensor that corresponds to the probability of whether an image is real or refined.
    """
    x = tf.keras.layers.Conv2D(96, (3, 3), activation="relu", strides=(2, 2), padding="same")(input_image_tensor)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", strides=(2, 2), padding="same")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding="same")(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", strides=(1, 1), padding="same")(x)
    x = tf.keras.layers.Conv2D(32, (1, 1), activation="relu", strides=(1, 1), padding="same")(x)
    x = tf.keras.layers.Conv2D(2, (1, 1), activation="relu", strides=(1, 1), padding="same")(x)

    # here one feature map corresponds to `is_real` and the other to `is_refined`,
    # and the custom loss function is then `tf.nn.sparse_softmax_cross_entropy_with_logits`
    return layers.Reshape((-1, 2))(x)

Combining Models: Adversarial training of refiner network Rθ and discriminator network Dφ and combining them into single loss-functions and models that can be trained

In [ ]:
#
# define model input and output tensors
#

synthetic_image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
refined_image_tensor = refiner_network(synthetic_image_tensor)

refined_or_real_image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
discriminator_output = discriminator_network(refined_or_real_image_tensor)

#
# define models
#

refiner_model = models.Model(synthetic_image_tensor, refined_image_tensor, name='refiner')
discriminator_model = models.Model(refined_or_real_image_tensor, discriminator_output,
                                   name='discriminator')

# combined must output the refined image along w/ the disc's classification of it for the refiner's self-reg loss
refiner_model_output = refiner_model(synthetic_image_tensor)
combined_output = discriminator_model(refiner_model_output)
combined_model = models.Model(synthetic_image_tensor, [refiner_model_output, combined_output],
                              name='combined')

discriminator_model_output_shape = discriminator_model.output_shape

print(refiner_model.summary())
print(discriminator_model.summary())
print(combined_model.summary())

In [ ]:
!pip uninstall pydot
!pip uninstall pydotplus
!pip uninstall graphviz

!pip install pydot
!pip install pydotplus



Creating requirements.txt

In [45]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [31]:
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
# Define model
try:
    model_to_dot(refiner_model, show_shapes=True).write_svg('refiner_model.svg')
    SVG('refiner_model.svg')
except ImportError:
    print('Not running the patched version of keras/pydot!')
    pass

In [34]:
try:
    model_to_dot(discriminator_model, show_shapes=True).write_svg('discriminator_model.svg')
    SVG('discriminator_model.svg')
except ImportError:
    pass

In [35]:
#
# define custom l1 loss function for the refiner
#

def self_regularization_loss(y_true, y_pred):
    delta = 0.0001  # FIXME: need to figure out an appropriate value for this
    return tf.multiply(delta, tf.reduce_sum(tf.abs(y_pred - y_true)))

#
# define custom local adversarial loss (softmax for each image section) for the discriminator
# the adversarial loss function is the sum of the cross-entropy losses over the local patches
#

def local_adversarial_loss(y_true, y_pred):
    # y_true and y_pred have shape (batch_size, # of local patches, 2), but really we just want to average over
    # the local patches and batch size so we can reshape to (batch_size * # of local patches, 2)
    y_true = tf.reshape(y_true, (-1, 2))
    y_pred = tf.reshape(y_pred, (-1, 2))
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

    return tf.reduce_mean(loss)

Compile Models: Here we combine the models and compile them with the loss functions defined above

In [36]:
from tensorflow.keras import optimizers
sgd = optimizers.SGD(lr=1e-3)

refiner_model.compile(optimizer=sgd, loss=self_regularization_loss)
discriminator_model.compile(optimizer=sgd, loss=local_adversarial_loss)
discriminator_model.trainable = False
combined_model.compile(optimizer=sgd, loss=[self_regularization_loss, local_adversarial_loss])

C:\Users\smoha\.conda\envs\env_py_3_8_15\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [37]:
refiner_model_path = None
discriminator_model_path = None

Data Generators: Here we setup the pipeline to feed new images to both models

In [38]:
datagen = image.ImageDataGenerator(
    preprocessing_function=applications.xception.preprocess_input,
    data_format='channels_last')

flow_from_directory_params = {'target_size': (img_height, img_width),
                              'color_mode': 'grayscale' if img_channels == 1 else 'rgb',
                              'class_mode': None,
                              'batch_size': batch_size}
flow_params = {'batch_size': batch_size}

synthetic_generator = datagen.flow(
    x = syn_image_stack,
    **flow_params
)

real_generator = datagen.flow(
    x = real_image_stack,
    **flow_params
)

def get_image_batch(generator):
    """keras generators may generate an incomplete batch for the last batch"""
    img_batch = generator.next()
    if len(img_batch) != batch_size:
        img_batch = generator.next()

    assert len(img_batch) == batch_size

    return img_batch

# the target labels for the cross-entropy loss layer are 0 for every yj (real) and 1 for every xi (refined)
y_real = np.array([[[1.0, 0.0]] * discriminator_model_output_shape[1]] * batch_size)
y_refined = np.array([[[0.0, 1.0]] * discriminator_model_output_shape[1]] * batch_size)
assert y_real.shape == (batch_size, discriminator_model_output_shape[1], 2)
batch_out = get_image_batch(synthetic_generator)
assert batch_out.shape == (batch_size, img_height, img_width, img_channels), "Image Dimensions do not match, {}!={}".format(batch_out.shape, (batch_size, img_height, img_width, img_channels))

In [39]:

#print(str(synthetic_image_batch.shape))
#refiner_model.predict_on_batch(synthetic_image_batch)
#print( str(refiner_model.predict_on_batch(synthetic_image_batch).shape) )
#synthetic_image_batch = get_image_batch(synthetic_generator)



Pretraining: Here we pretraining the models before we start the full-blown training procedure

In [40]:
if not refiner_model_path:
    # we first train the Rθ network with just self-regularization loss for 1,000 steps
    print('pre-training the refiner network...')
    gen_loss = np.zeros(shape=len(refiner_model.metrics_names))

    for i in range(pre_steps):
        synthetic_image_batch = get_image_batch(synthetic_generator)
        gen_loss = np.add(refiner_model.train_on_batch(synthetic_image_batch, synthetic_image_batch), gen_loss)

        # log every `log_interval` steps
        if not i % log_interval:
            figure_name = 'refined_image_batch_pre_train_step_{}.png'.format(i)
            print('Saving batch of refined images during pre-training at step: {}.'.format(i))

            synthetic_image_batch = get_image_batch(synthetic_generator)
            plot_batch(
                np.concatenate( (synthetic_image_batch, refiner_model.predict_on_batch(synthetic_image_batch)) ),
                os.path.join(cache_dir, figure_name),
                label_batch=['Synthetic'] * batch_size + ['Refined'] * batch_size)

            print('Refiner model self regularization loss: {}.'.format(gen_loss / log_interval))
            gen_loss = np.zeros(shape=len(refiner_model.metrics_names))

    refiner_model.save(os.path.join(cache_dir, 'refiner_model_pre_trained.h5'))
else:
    refiner_model.load_weights(refiner_model_path)

if not discriminator_model_path:
    # and Dφ for 200 steps (one mini-batch for refined images, another for real)
    print('pre-training the discriminator network...')
    disc_loss = np.zeros(shape=len(discriminator_model.metrics_names))

    for _ in range(pre_steps):
        real_image_batch = get_image_batch(real_generator)
        disc_loss = np.add(discriminator_model.train_on_batch(real_image_batch, y_real), disc_loss)

        synthetic_image_batch = get_image_batch(synthetic_generator)
        refined_image_batch = refiner_model.predict_on_batch(synthetic_image_batch)
        disc_loss = np.add(discriminator_model.train_on_batch(refined_image_batch, y_refined), disc_loss)

    discriminator_model.save(os.path.join(cache_dir, 'discriminator_model_pre_trained.h5'))

    # hard-coded for now
    print('Discriminator model loss: {}.'.format(disc_loss / (100 * 2)))
else:
    discriminator_model.load_weights(discriminator_model_path)

pre-training the refiner network...
Saving batch of refined images during pre-training at step: 0.
Refiner model self regularization loss: [].
pre-training the discriminator network...
Discriminator model loss: [].


Full Training: Here we run the full training of the model (normally for thousands of iterations, but here just a few). The images are saved in the output directory for examining the training

In [44]:
## Run this cell two times
# TODO: what is an appropriate size for the image history buffer?
image_history_buffer = ImageHistoryBuffer((0, img_height, img_width, img_channels), batch_size * 100, batch_size)

combined_loss = np.zeros(shape=len(combined_model.metrics_names))
disc_loss_real = np.zeros(shape=len(discriminator_model.metrics_names))
disc_loss_refined = np.zeros(shape=len(discriminator_model.metrics_names))

# see Algorithm 1 in https://arxiv.org/pdf/1612.07828v1.pdf
for i in range(nb_steps):
    print('Step: {} of {}.'.format(i, nb_steps))

    # train the refiner
    for _ in range(k_g * 2):
        # sample a mini-batch of synthetic images
        synthetic_image_batch = get_image_batch(synthetic_generator)

        # update θ by taking an SGD step on mini-batch loss LR(θ)
        combined_loss = np.add(combined_model.train_on_batch(synthetic_image_batch,
                                                             [synthetic_image_batch, y_real]), combined_loss)

    for _ in range(k_d):
        # sample a mini-batch of synthetic and real images
        synthetic_image_batch = get_image_batch(synthetic_generator)
        real_image_batch = get_image_batch(real_generator)

        # refine the synthetic images w/ the current refiner
        refined_image_batch = refiner_model.predict_on_batch(synthetic_image_batch)

        # use a history of refined images
        half_batch_from_image_history = image_history_buffer.get_from_image_history_buffer()
        image_history_buffer.add_to_image_history_buffer(refined_image_batch)

        if len(half_batch_from_image_history):
            refined_image_batch[:batch_size // 2] = half_batch_from_image_history

        # update φ by taking an SGD step on mini-batch loss LD(φ)
        disc_loss_real = np.add(discriminator_model.train_on_batch(real_image_batch, y_real), disc_loss_real)
        disc_loss_refined = np.add(discriminator_model.train_on_batch(refined_image_batch, y_refined),
                                   disc_loss_refined)

    if not i % log_interval:
        # plot batch of refined images w/ current refiner
        figure_name = 'refined_image_batch_step_{}.png'.format(i)
        print('Saving batch of refined images at adversarial step: {}.'.format(i))

        synthetic_image_batch = get_image_batch(synthetic_generator)
        plot_batch(
            np.concatenate((synthetic_image_batch, refiner_model.predict_on_batch(synthetic_image_batch))),
            os.path.join(cache_dir, figure_name),
            label_batch=['Synthetic'] * batch_size + ['Refined'] * batch_size)

        # log loss summary
        print('Refiner model loss: {}.'.format(combined_loss / (log_interval * k_g * 2)))
        print('Discriminator model loss real: {}.'.format(disc_loss_real / (log_interval * k_d * 2)))
        print('Discriminator model loss refined: {}.'.format(disc_loss_refined / (log_interval * k_d * 2)))

        combined_loss = np.zeros(shape=len(combined_model.metrics_names))
        disc_loss_real = np.zeros(shape=len(discriminator_model.metrics_names))
        disc_loss_refined = np.zeros(shape=len(discriminator_model.metrics_names))

        # save model checkpoints
        model_checkpoint_base_name = os.path.join(cache_dir, '{}_model_step_{}.h5')
        refiner_model.save(model_checkpoint_base_name.format('refiner', i))
        discriminator_model.save(model_checkpoint_base_name.format('discriminator', i))

Step: 0 of 10000.
Saving batch of refined images at adversarial step: 0.
Refiner model loss: [0.01298325 0.0061376  0.00684565].
Discriminator model loss real: [0.00338777].
Discriminator model loss refined: [0.00351322].
Step: 1 of 10000.
Step: 2 of 10000.
Step: 3 of 10000.
Step: 4 of 10000.
Step: 5 of 10000.
Step: 6 of 10000.
Step: 7 of 10000.
Step: 8 of 10000.
Step: 9 of 10000.
Step: 10 of 10000.
Step: 11 of 10000.
Step: 12 of 10000.
Step: 13 of 10000.
Step: 14 of 10000.
Step: 15 of 10000.
Step: 16 of 10000.
Step: 17 of 10000.
Step: 18 of 10000.
Step: 19 of 10000.
Step: 20 of 10000.
Step: 21 of 10000.
Step: 22 of 10000.
Step: 23 of 10000.
Step: 24 of 10000.
Step: 25 of 10000.
Step: 26 of 10000.
Step: 27 of 10000.
Step: 28 of 10000.
Step: 29 of 10000.
Step: 30 of 10000.
Step: 31 of 10000.
Step: 32 of 10000.
Step: 33 of 10000.
Step: 34 of 10000.
Step: 35 of 10000.
Step: 36 of 10000.
Step: 37 of 10000.
Step: 38 of 10000.
Step: 39 of 10000.
Step: 40 of 10000.
Step: 41 of 10000.
Step: 42

Step: 375 of 10000.
Step: 376 of 10000.
Step: 377 of 10000.
Step: 378 of 10000.
Step: 379 of 10000.
Step: 380 of 10000.
Step: 381 of 10000.
Step: 382 of 10000.
Step: 383 of 10000.
Step: 384 of 10000.
Step: 385 of 10000.
Step: 386 of 10000.
Step: 387 of 10000.
Step: 388 of 10000.
Step: 389 of 10000.
Step: 390 of 10000.
Step: 391 of 10000.
Step: 392 of 10000.
Step: 393 of 10000.
Step: 394 of 10000.
Step: 395 of 10000.
Step: 396 of 10000.
Step: 397 of 10000.
Step: 398 of 10000.
Step: 399 of 10000.
Step: 400 of 10000.
Saving batch of refined images at adversarial step: 400.
Refiner model loss: [0.90960506 0.2263764  0.68322866].
Discriminator model loss real: [0.3343539].
Discriminator model loss refined: [0.35213755].
Step: 401 of 10000.
Step: 402 of 10000.
Step: 403 of 10000.
Step: 404 of 10000.
Step: 405 of 10000.
Step: 406 of 10000.
Step: 407 of 10000.
Step: 408 of 10000.
Step: 409 of 10000.
Step: 410 of 10000.
Step: 411 of 10000.
Step: 412 of 10000.
Step: 413 of 10000.
Step: 414 of 10

Step: 744 of 10000.
Step: 745 of 10000.
Step: 746 of 10000.
Step: 747 of 10000.
Step: 748 of 10000.
Step: 749 of 10000.
Step: 750 of 10000.
Step: 751 of 10000.
Step: 752 of 10000.
Step: 753 of 10000.
Step: 754 of 10000.
Step: 755 of 10000.
Step: 756 of 10000.
Step: 757 of 10000.
Step: 758 of 10000.
Step: 759 of 10000.
Step: 760 of 10000.
Step: 761 of 10000.
Step: 762 of 10000.
Step: 763 of 10000.
Step: 764 of 10000.
Step: 765 of 10000.
Step: 766 of 10000.
Step: 767 of 10000.
Step: 768 of 10000.
Step: 769 of 10000.
Step: 770 of 10000.
Step: 771 of 10000.
Step: 772 of 10000.
Step: 773 of 10000.
Step: 774 of 10000.
Step: 775 of 10000.
Step: 776 of 10000.
Step: 777 of 10000.
Step: 778 of 10000.
Step: 779 of 10000.
Step: 780 of 10000.
Step: 781 of 10000.
Step: 782 of 10000.
Step: 783 of 10000.
Step: 784 of 10000.
Step: 785 of 10000.
Step: 786 of 10000.
Step: 787 of 10000.
Step: 788 of 10000.
Step: 789 of 10000.
Step: 790 of 10000.
Step: 791 of 10000.
Step: 792 of 10000.
Step: 793 of 10000.


Step: 1107 of 10000.
Step: 1108 of 10000.
Step: 1109 of 10000.
Step: 1110 of 10000.
Step: 1111 of 10000.
Step: 1112 of 10000.
Step: 1113 of 10000.
Step: 1114 of 10000.
Step: 1115 of 10000.
Step: 1116 of 10000.
Step: 1117 of 10000.
Step: 1118 of 10000.
Step: 1119 of 10000.
Step: 1120 of 10000.
Step: 1121 of 10000.
Step: 1122 of 10000.
Step: 1123 of 10000.
Step: 1124 of 10000.
Step: 1125 of 10000.
Step: 1126 of 10000.
Step: 1127 of 10000.
Step: 1128 of 10000.
Step: 1129 of 10000.
Step: 1130 of 10000.
Step: 1131 of 10000.
Step: 1132 of 10000.
Step: 1133 of 10000.
Step: 1134 of 10000.
Step: 1135 of 10000.
Step: 1136 of 10000.
Step: 1137 of 10000.
Step: 1138 of 10000.
Step: 1139 of 10000.
Step: 1140 of 10000.
Step: 1141 of 10000.
Step: 1142 of 10000.
Step: 1143 of 10000.
Step: 1144 of 10000.
Step: 1145 of 10000.
Step: 1146 of 10000.
Step: 1147 of 10000.
Step: 1148 of 10000.
Step: 1149 of 10000.
Step: 1150 of 10000.
Step: 1151 of 10000.
Step: 1152 of 10000.
Step: 1153 of 10000.
Step: 1154 of

Step: 1468 of 10000.
Step: 1469 of 10000.
Step: 1470 of 10000.
Step: 1471 of 10000.
Step: 1472 of 10000.
Step: 1473 of 10000.
Step: 1474 of 10000.
Step: 1475 of 10000.
Step: 1476 of 10000.
Step: 1477 of 10000.
Step: 1478 of 10000.
Step: 1479 of 10000.
Step: 1480 of 10000.
Step: 1481 of 10000.
Step: 1482 of 10000.
Step: 1483 of 10000.
Step: 1484 of 10000.
Step: 1485 of 10000.
Step: 1486 of 10000.
Step: 1487 of 10000.
Step: 1488 of 10000.
Step: 1489 of 10000.
Step: 1490 of 10000.
Step: 1491 of 10000.
Step: 1492 of 10000.
Step: 1493 of 10000.
Step: 1494 of 10000.
Step: 1495 of 10000.
Step: 1496 of 10000.
Step: 1497 of 10000.
Step: 1498 of 10000.
Step: 1499 of 10000.
Step: 1500 of 10000.
Saving batch of refined images at adversarial step: 1500.
Refiner model loss: [0.81808237 0.12423063 0.69385174].
Discriminator model loss real: [0.31221253].
Discriminator model loss refined: [0.34878434].
Step: 1501 of 10000.
Step: 1502 of 10000.
Step: 1503 of 10000.
Step: 1504 of 10000.
Step: 1505 of 10

...

Step: 8218 of 10000.
Step: 8219 of 10000.
Step: 8220 of 10000.
Step: 8221 of 10000.
Step: 8222 of 10000.
Step: 8223 of 10000.
Step: 8224 of 10000.
Step: 8225 of 10000.
Step: 8226 of 10000.
Step: 8227 of 10000.
Step: 8228 of 10000.
Step: 8229 of 10000.
Step: 8230 of 10000.
Step: 8231 of 10000.
Step: 8232 of 10000.
Step: 8233 of 10000.
Step: 8234 of 10000.
Step: 8235 of 10000.
Step: 8236 of 10000.
Step: 8237 of 10000.
Step: 8238 of 10000.
Step: 8239 of 10000.
Step: 8240 of 10000.
Step: 8241 of 10000.
Step: 8242 of 10000.
Step: 8243 of 10000.
Step: 8244 of 10000.
Step: 8245 of 10000.
Step: 8246 of 10000.
Step: 8247 of 10000.
Step: 8248 of 10000.
Step: 8249 of 10000.
Step: 8250 of 10000.
Step: 8251 of 10000.
Step: 8252 of 10000.
Step: 8253 of 10000.
Step: 8254 of 10000.
Step: 8255 of 10000.
Step: 8256 of 10000.
Step: 8257 of 10000.
Step: 8258 of 10000.
Step: 8259 of 10000.
Step: 8260 of 10000.
Step: 8261 of 10000.
Step: 8262 of 10000.
Step: 8263 of 10000.
Step: 8264 of 10000.
Step: 8265 of

Step: 8579 of 10000.
Step: 8580 of 10000.
Step: 8581 of 10000.
Step: 8582 of 10000.
Step: 8583 of 10000.
Step: 8584 of 10000.
Step: 8585 of 10000.
Step: 8586 of 10000.
Step: 8587 of 10000.
Step: 8588 of 10000.
Step: 8589 of 10000.
Step: 8590 of 10000.
Step: 8591 of 10000.
Step: 8592 of 10000.
Step: 8593 of 10000.
Step: 8594 of 10000.
Step: 8595 of 10000.
Step: 8596 of 10000.
Step: 8597 of 10000.
Step: 8598 of 10000.
Step: 8599 of 10000.
Step: 8600 of 10000.
Saving batch of refined images at adversarial step: 8600.
Refiner model loss: [1.60838556 0.09740153 1.51098403].
Discriminator model loss real: [0.21800484].
Discriminator model loss refined: [0.24216126].
Step: 8601 of 10000.
Step: 8602 of 10000.
Step: 8603 of 10000.
Step: 8604 of 10000.
Step: 8605 of 10000.
Step: 8606 of 10000.
Step: 8607 of 10000.
Step: 8608 of 10000.
Step: 8609 of 10000.
Step: 8610 of 10000.
Step: 8611 of 10000.
Step: 8612 of 10000.
Step: 8613 of 10000.
Step: 8614 of 10000.
Step: 8615 of 10000.
Step: 8616 of 10

Step: 8930 of 10000.
Step: 8931 of 10000.
Step: 8932 of 10000.
Step: 8933 of 10000.
Step: 8934 of 10000.
Step: 8935 of 10000.
Step: 8936 of 10000.
Step: 8937 of 10000.
Step: 8938 of 10000.
Step: 8939 of 10000.
Step: 8940 of 10000.
Step: 8941 of 10000.
Step: 8942 of 10000.
Step: 8943 of 10000.
Step: 8944 of 10000.
Step: 8945 of 10000.
Step: 8946 of 10000.
Step: 8947 of 10000.
Step: 8948 of 10000.
Step: 8949 of 10000.
Step: 8950 of 10000.
Step: 8951 of 10000.
Step: 8952 of 10000.
Step: 8953 of 10000.
Step: 8954 of 10000.
Step: 8955 of 10000.
Step: 8956 of 10000.
Step: 8957 of 10000.
Step: 8958 of 10000.
Step: 8959 of 10000.
Step: 8960 of 10000.
Step: 8961 of 10000.
Step: 8962 of 10000.
Step: 8963 of 10000.
Step: 8964 of 10000.
Step: 8965 of 10000.
Step: 8966 of 10000.
Step: 8967 of 10000.
Step: 8968 of 10000.
Step: 8969 of 10000.
Step: 8970 of 10000.
Step: 8971 of 10000.
Step: 8972 of 10000.
Step: 8973 of 10000.
Step: 8974 of 10000.
Step: 8975 of 10000.
Step: 8976 of 10000.
Step: 8977 of

Step: 9291 of 10000.
Step: 9292 of 10000.
Step: 9293 of 10000.
Step: 9294 of 10000.
Step: 9295 of 10000.
Step: 9296 of 10000.
Step: 9297 of 10000.
Step: 9298 of 10000.
Step: 9299 of 10000.
Step: 9300 of 10000.
Saving batch of refined images at adversarial step: 9300.
Refiner model loss: [1.69512682 0.12953069 1.56559613].
Discriminator model loss real: [0.21771386].
Discriminator model loss refined: [0.24119016].
Step: 9301 of 10000.
Step: 9302 of 10000.
Step: 9303 of 10000.
Step: 9304 of 10000.
Step: 9305 of 10000.
Step: 9306 of 10000.
Step: 9307 of 10000.
Step: 9308 of 10000.
Step: 9309 of 10000.
Step: 9310 of 10000.
Step: 9311 of 10000.
Step: 9312 of 10000.
Step: 9313 of 10000.
Step: 9314 of 10000.
Step: 9315 of 10000.
Step: 9316 of 10000.
Step: 9317 of 10000.
Step: 9318 of 10000.
Step: 9319 of 10000.
Step: 9320 of 10000.
Step: 9321 of 10000.
Step: 9322 of 10000.
Step: 9323 of 10000.
Step: 9324 of 10000.
Step: 9325 of 10000.
Step: 9326 of 10000.
Step: 9327 of 10000.
Step: 9328 of 10

Step: 9642 of 10000.
Step: 9643 of 10000.
Step: 9644 of 10000.
Step: 9645 of 10000.
Step: 9646 of 10000.
Step: 9647 of 10000.
Step: 9648 of 10000.
Step: 9649 of 10000.
Step: 9650 of 10000.
Step: 9651 of 10000.
Step: 9652 of 10000.
Step: 9653 of 10000.
Step: 9654 of 10000.
Step: 9655 of 10000.
Step: 9656 of 10000.
Step: 9657 of 10000.
Step: 9658 of 10000.
Step: 9659 of 10000.
Step: 9660 of 10000.
Step: 9661 of 10000.
Step: 9662 of 10000.
Step: 9663 of 10000.
Step: 9664 of 10000.
Step: 9665 of 10000.
Step: 9666 of 10000.
Step: 9667 of 10000.
Step: 9668 of 10000.
Step: 9669 of 10000.
Step: 9670 of 10000.
Step: 9671 of 10000.
Step: 9672 of 10000.
Step: 9673 of 10000.
Step: 9674 of 10000.
Step: 9675 of 10000.
Step: 9676 of 10000.
Step: 9677 of 10000.
Step: 9678 of 10000.
Step: 9679 of 10000.
Step: 9680 of 10000.
Step: 9681 of 10000.
Step: 9682 of 10000.
Step: 9683 of 10000.
Step: 9684 of 10000.
Step: 9685 of 10000.
Step: 9686 of 10000.
Step: 9687 of 10000.
Step: 9688 of 10000.
Step: 9689 of